In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

In [2]:
import pandas as pd
from pprint import pprint
from llm_data_quality_assistant.llm_integration import combine_results_1
from llm_data_quality_assistant.pipeline import Pipeline

# Read the dataframes
# LLM cleaned
llm_path = "../analysis/repairs/allergen/merged_dataset_gemini_2_0_flash_lite_50_rows_context.csv"
df_llm = pd.read_csv(llm_path)

# Parker cleaned
parker_path = "../datasets/parker_datasets/allergen/parker-repair/allergen_repair.csv"
df_parker = pd.read_csv(parker_path)

# Original corrupted
orig_path = "../datasets/parker_datasets/allergen/allergen_corrupted_first1000.csv"
df_original = pd.read_csv(orig_path)

# Check if all DataFrames have the same shape
if not (df_llm.shape == df_parker.shape == df_original.shape):
    raise ValueError("Not all DataFrames have the same shape.")

# Ensure all DataFrames have the same order of 'code'
if "code" not in df_parker.columns or "code" not in df_llm.columns or "code" not in df_original.columns:
    raise ValueError("All DataFrames must contain a 'code' column.")

# Ensure all DataFrames have the same column order
column_order = df_parker.columns.tolist()
df_llm = df_llm[column_order]
df_original = df_original[column_order]


df_parker = df_parker.sort_values("code").reset_index(drop=True)
df_llm = df_llm.sort_values("code").reset_index(drop=True)
df_original = df_original.sort_values("code").reset_index(drop=True)

if not (
    (df_parker["code"].tolist() == df_llm["code"].tolist() == df_original["code"].tolist())
):
    raise ValueError("The 'code' column does not have the same order in all DataFrames.")


# Calculate the percentage of differing cells between df_parker and df_llm (excluding 'code' column)
diff_mask = (df_parker[column_order].values != df_llm[column_order].values)
num_diff = diff_mask.sum()
total_cells = diff_mask.size
percent_diff = (num_diff / total_cells) * 100

print(f"Percentage of differing cells between df_parker and df_llm (excluding 'code'): {percent_diff:.2f}%")


# Combine results
combined_df = combine_results_1(df_llm=df_llm, df_parker=df_parker, df_original=df_original)

# Show the result
combined_df.head()


Percentage of differing cells between df_parker and df_llm (excluding 'code'): 0.31%


,eggs,gluten,hazelnut,pistachio,code,lupin,crustaceans,milk,sulfite,walnut,...,nuts,fish,molluscs,almondnuts,cashew,peanut,sesame,soy,brazil_nuts,macadamia_nuts
0,0,2,0,0,42239499,0,0,0,0,0,...,2,0,0,0,0,0,0,2,0,0
1,0,2,0,0,42239499,0,0,0,0,0,...,2,0,0,0,0,0,0,2,0,0
2,0,0,0,0,42256199,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,42256199,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,2,0,0,42330660,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:

gold_standard_path = "../datasets/parker_datasets/allergen/allergen_cleaned_gold_first1000.csv"
gold_standard = pd.read_csv(gold_standard_path)
# Ensure gold_standard has the same column order and row order as combined_df
gold_standard = gold_standard[column_order]
gold_standard = gold_standard.sort_values("code").reset_index(drop=True)


# cleaned_dataset = combined_df (already defined)
# corrupted_dataset = df_original (already defined)

micro_eval = Pipeline.evaluate_micro(
    gold_standard=gold_standard,
    cleaned_dataset=combined_df,
    corrupted_dataset=df_original
)


print("Micro evaluation result:")
pprint(micro_eval)

Micro evaluation result:
{'accuracy': 0.9382171226831421,
 'column_names': ['eggs',
                  'gluten',
                  'hazelnut',
                  'pistachio',
                  'code',
                  'lupin',
                  'crustaceans',
                  'milk',
                  'sulfite',
                  'walnut',
                  'mustard',
                  'celery',
                  'nuts',
                  'fish',
                  'molluscs',
                  'almondnuts',
                  'cashew',
                  'peanut',
                  'sesame',
                  'soy',
                  'brazil_nuts',
                  'macadamia_nuts'],
 'f1_score': 0.43319838056680166,
 'false_negative': 251,
 'false_negative_rate': 0.7011173184357542,
 'false_positive': 29,
 'false_positive_rate': 0.006947771921418304,
 'num_columns': 22,
 'num_rows': 206,
 'precision': 0.7867647058823529,
 'recall': 0.2988826815642458,
 'true_negative': 4145,
 'true_pos

In [4]:
import pandas as pd
from llm_data_quality_assistant.llm_integration import combine_results_1

# Create three simple DataFrames
df_parker = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': ['x', 'y', 'z', 'w', 'v'],
    'C': [True, False, True, False, True]
})

df_llm = pd.DataFrame({
    'A': [1, 2, 0, 4, 5],
    'B': ['x', 'y', 'z', 'wrong', 'v'],
    'C': [True, False, False, False, True]
})

df_original = pd.DataFrame({
    'A': [0, 2, 3, 4, 0],
    'B': ['x', 'wrong', 'z', 'w', 'v'],
    'C': [False, False, True, True, True]
})

# Print the input DataFrames
print("df_parker:")
print(df_parker)
print("\ndf_llm:")
print(df_llm)
print("\ndf_original:")
print(df_original)

# Invoke combine_results_1
result = combine_results_1(df_parker, df_llm, df_original)

# Print the result
print("\nResult of combine_results_1:")
print(result)

df_parker:
   A  B      C
0  1  x   True
1  2  y  False
2  3  z   True
3  4  w  False
4  5  v   True

df_llm:
   A      B      C
0  1      x   True
1  2      y  False
2  0      z  False
3  4  wrong  False
4  5      v   True

df_original:
   A      B      C
0  0      x  False
1  2  wrong  False
2  3      z   True
3  4      w   True
4  0      v   True

Result of combine_results_1:
   A  B      C
0  1  x   True
1  2  y  False
2  3  z   True
3  4  w  False
4  5  v   True
